### RAG Pipelines- Data Ingestion to Vector DB Pipeline


In [110]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [111]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: 714022201098_SIVRAAMKRISHNAN.pdf
  ✓ Loaded 2 pages

Processing: FTH-Quaterly-Insight-Sep-2025.pdf
  ✓ Loaded 20 pages

Processing: SivRaamKrishnan.pdf
  ✓ Loaded 1 pages

Total documents loaded: 23


In [112]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [113]:
chunks=split_documents(all_pdf_documents)
chunks

Split 23 documents into 76 chunks

Example chunk:
Content: SIV RAAM KRISHNAN .K.V 
+91-8883711103 | sivraamkrishnankv@gmail.com | https://github.com/sivraamkrishannkv | 
https://www.linkedin.com/in/sivraamkrishnankv/ | https://leetcode.com/sivraamkrishnankv/ ...
Metadata: {'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-10T13:47:30+05:30', 'author': 'SIV RAAM KRISHNAN.K.V', 'moddate': '2025-08-10T13:47:30+05:30', 'title': 'Microsoft Word - goML_ML engineer.docx', 'source': '..\\data\\pdf\\714022201098_SIVRAAMKRISHNAN.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': '714022201098_SIVRAAMKRISHNAN.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-10T13:47:30+05:30', 'author': 'SIV RAAM KRISHNAN.K.V', 'moddate': '2025-08-10T13:47:30+05:30', 'title': 'Microsoft Word - goML_ML engineer.docx', 'source': '..\\data\\pdf\\714022201098_SIVRAAMKRISHNAN.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': '714022201098_SIVRAAMKRISHNAN.pdf', 'file_type': 'pdf'}, page_content="SIV RAAM KRISHNAN .K.V \n+91-8883711103 | sivraamkrishnankv@gmail.com | https://github.com/sivraamkrishannkv | \nhttps://www.linkedin.com/in/sivraamkrishnankv/ | https://leetcode.com/sivraamkrishnankv/   \nPROFILE \n \n \n A skilled AI/ML engineer who can develop and deploy scalable ML models with optimized pipelines. Proficient in \nOpenCV, Docker, Git, and CI/CD workflows. Has hands-on experience in deploying and monitoring machine learning models. \nStrong understanding of software engineering principles, focusing on performance, reliability, and

In [114]:
all_pdf_documents


[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-10T13:47:30+05:30', 'author': 'SIV RAAM KRISHNAN.K.V', 'moddate': '2025-08-10T13:47:30+05:30', 'title': 'Microsoft Word - goML_ML engineer.docx', 'source': '..\\data\\pdf\\714022201098_SIVRAAMKRISHNAN.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': '714022201098_SIVRAAMKRISHNAN.pdf', 'file_type': 'pdf'}, page_content="SIV RAAM KRISHNAN .K.V \n+91-8883711103 | sivraamkrishnankv@gmail.com | https://github.com/sivraamkrishannkv | \nhttps://www.linkedin.com/in/sivraamkrishnankv/ | https://leetcode.com/sivraamkrishnankv/   \nPROFILE \n \n \n A skilled AI/ML engineer who can develop and deploy scalable ML models with optimized pipelines. Proficient in \nOpenCV, Docker, Git, and CI/CD workflows. Has hands-on experience in deploying and monitoring machine learning models. \nStrong understanding of software engineering principles, focusing on performance, reliability, and

### embedding And vectorStoreDB

In [115]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [116]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore


In [117]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 169


In [118]:
chunks

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-10T13:47:30+05:30', 'author': 'SIV RAAM KRISHNAN.K.V', 'moddate': '2025-08-10T13:47:30+05:30', 'title': 'Microsoft Word - goML_ML engineer.docx', 'source': '..\\data\\pdf\\714022201098_SIVRAAMKRISHNAN.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': '714022201098_SIVRAAMKRISHNAN.pdf', 'file_type': 'pdf'}, page_content="SIV RAAM KRISHNAN .K.V \n+91-8883711103 | sivraamkrishnankv@gmail.com | https://github.com/sivraamkrishannkv | \nhttps://www.linkedin.com/in/sivraamkrishnankv/ | https://leetcode.com/sivraamkrishnankv/   \nPROFILE \n \n \n A skilled AI/ML engineer who can develop and deploy scalable ML models with optimized pipelines. Proficient in \nOpenCV, Docker, Git, and CI/CD workflows. Has hands-on experience in deploying and monitoring machine learning models. \nStrong understanding of software engineering principles, focusing on performance, reliability, and

### Convert the text to embeddings

In [119]:
texts=[doc.page_content for doc in chunks]
print((texts))
## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

["SIV RAAM KRISHNAN .K.V \n+91-8883711103 | sivraamkrishnankv@gmail.com | https://github.com/sivraamkrishannkv | \nhttps://www.linkedin.com/in/sivraamkrishnankv/ | https://leetcode.com/sivraamkrishnankv/   \nPROFILE \n \n \n A skilled AI/ML engineer who can develop and deploy scalable ML models with optimized pipelines. Proficient in \nOpenCV, Docker, Git, and CI/CD workflows. Has hands-on experience in deploying and monitoring machine learning models. \nStrong understanding of software engineering principles, focusing on performance, reliability, and maintainability. Capable of \ncollaborating across teams and delivering high-quality, production-ready solutions. \nEDUCATION \n \n \nSri Shakthi Institute of Engineering & Technology, Coimbatore  \nB. Tech, Artificial Intelligence & Data Science Aug 2022 - May 2026 \n• CGPA: 8.19 \nChandra Matriculation Higher Secondary School, Coimbatore  \nHSC  May 2021 - Jun 2022 \n• Percentage: 78% \nSt.Joseph's Matriculation Higher Secondary School,

Batches: 100%|██████████| 3/3 [00:04<00:00,  1.40s/it]

Generated embeddings with shape: (76, 384)
Adding 76 documents to vector store...
Successfully added 76 documents to vector store
Total documents in collection: 245


## Retriever Pipeline from VectorStore


In [120]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = -0.5) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [121]:

rag_retriever

In [122]:
rag_retriever.retrieve("who is sivraam")

Retrieving documents for query: 'who is sivraam'
Top K: 5, Score threshold: -0.5
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 37.05it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_eff975e6_0',
  'content': "SIV RAAM KRISHNAN .K.V \n+91-8883711103 | sivraamkrishnankv@gmail.com | https://github.com/sivraamkrishannkv | \nhttps://www.linkedin.com/in/sivraamkrishnankv/ | https://leetcode.com/sivraamkrishnankv/   \nPROFILE \n \n \n A skilled AI/ML engineer who can develop and deploy scalable ML models with optimized pipelines. Proficient in \nOpenCV, Docker, Git, and CI/CD workflows. Has hands-on experience in deploying and monitoring machine learning models. \nStrong understanding of software engineering principles, focusing on performance, reliability, and maintainability. Capable of \ncollaborating across teams and delivering high-quality, production-ready solutions. \nEDUCATION \n \n \nSri Shakthi Institute of Engineering & Technology, Coimbatore  \nB. Tech, Artificial Intelligence & Data Science Aug 2022 - May 2026 \n• CGPA: 8.19 \nChandra Matriculation Higher Secondary School, Coimbatore  \nHSC  May 2021 - Jun 2022 \n• Percentage: 78% \nSt.Joseph's 

In [123]:
# Debug: Check what ChromaDB actually returns
query_embedding = embedding_manager.generate_embeddings(["who is sivraam"])[0]
results = vectorstore.collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=5
)

print("Full results structure:")
print(f"Keys: {results.keys()}")
print(f"Documents: {results.get('documents')}")
print(f"Metadatas: {results.get('metadatas')}")
print(f"Distances: {results.get('distances')}")
print(f"IDs: {results.get('ids')}")
print(f"\nCollection count: {vectorstore.collection.count()}")

Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 41.91it/s]

Generated embeddings with shape: (1, 384)
Full results structure:
Keys: dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas', 'distances'])
Documents: [["SIV RAAM KRISHNAN .K.V \n+91-8883711103 | sivraamkrishnankv@gmail.com | https://github.com/sivraamkrishannkv | \nhttps://www.linkedin.com/in/sivraamkrishnankv/ | https://leetcode.com/sivraamkrishnankv/   \nPROFILE \n \n \n A skilled AI/ML engineer who can develop and deploy scalable ML models with optimized pipelines. Proficient in \nOpenCV, Docker, Git, and CI/CD workflows. Has hands-on experience in deploying and monitoring machine learning models. \nStrong understanding of software engineering principles, focusing on performance, reliability, and maintainability. Capable of \ncollaborating across teams and delivering high-quality, production-ready solutions. \nEDUCATION \n \n \nSri Shakthi Institute of Engineering & Technology, Coimbatore  \nB. Tech, Artificial Intelligence & Data Science Aug 2022 - 